# 2. Update data daily

In [1]:
import pickle
import pandas as pd
import hopsworks
import os

from keys import TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3
from TomTomAPI import get_traffic_map_from_grid
from HolidaysnWeather import get_weather, is_holiday

In [2]:
coordinates = 59.34318, 18.05141 # Stockholm near Odenplan
zoom = 20

TOMTOM_keys = [TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3]

### 2.1. Collect traffic data

In [3]:
# Getting grid variable from memory
with open('variables/grid.pickle', 'rb') as file:
    # Deserialize and retrieve grid from pickle file
    grid = pickle.load(file)

In [32]:
# Collecting traffic flow from grid
success = False
i = 0
while i < len(TOMTOM_keys) and not success:
    try:
        traffic_map = get_traffic_map_from_grid(TOMTOM_keys[i],grid, zoom = zoom)
        success = True
    except:
        print('Failed with key:', TOMTOM_keys[i])
        i += 1

Failed with key: zJT5UBCDtRKSWwsbGp6LWjcQcIi0IcdV
num of requests: 298


In [71]:
from shapely.geometry import LineString

traffic_df = pd.DataFrame(traffic_map)
traffic_df['coordinates'] = traffic_df['coordinates'].apply(lambda x: LineString([(point['longitude'], point['latitude']) for point in x['coordinate']]))
traffic_df = traffic_df.drop_duplicates()

multi_index = pd.MultiIndex.from_arrays(
    [traffic_df['frc'], traffic_df['coordinates']],  # Pass as a list of arrays
    names=['frc', 'coordinates']  # Assign level names
)
traffic_df.index = multi_index

traffic_df = traffic_df.drop(columns = ['@version','frc', 'coordinates'])

print(traffic_df.shape)
traffic_df.head(1)

(179, 6)


,,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure
frc,coordinates,,,,,,
FRC6,"LINESTRING (18.047361336193802 59.342010331133054, 18.047573230706064 59.34179577386307, 18.04913561745761 59.34021728308633, 18.049217424832563 59.34013278083443, 18.049815557443026 59.339518562397586, 18.04989736481798 59.3394354262025, 18.050928674184462 59.33839203896625, 18.05185805960835 59.33744924414018, 18.051886222803006 59.33741974932061, 18.05198948785008 59.33730442005571, 18.0520189921493 59.33727625024616, 18.05265199347693 59.336659329834355, 18.052741847478956 59.336566782256796, 18.05288936897483 59.33634150372781, 18.052956424200204 59.33624626218505, 18.05304761930674 59.33612019924765)",23,23,114,114,1.0,False


### 2.2. Collect weather data

In [8]:
weather_df = get_weather(coordinates)
weather_df

,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
2024-12-17,3.876,-0.024,0.0,33.48,279.494843


### 2.3. Collect weather data

In [9]:
holiday_status = is_holiday()
weather_df['holiday_status'] = holiday_status
weather_df

,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,holiday_status
2024-12-17,3.876,-0.024,0.0,33.48,279.494843,0


### 2.4. Joining data and uploading to Hopsworks

In [ ]:
# Get the API key from GitHub Secrets
HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')

# Get AQI API KEY from secrets of hopsworks
proj = hopsworks.login(project="ID2223LAB1KTH")
conn = hopsworks.connection(host="c.app.hopsworks.ai", project=proj, api_key_value=os.environ.get('HOPSWORKS_API_KEY'))
secrets = conn.get_secrets_api()

AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

In [ ]:
# Initialize the feature store
fs = proj.get_feature_store()

# Define and insert the Forecast Weather Feature Group
feature_group_name = "stockholm_weather"
version = 1
fg = fs.get_feature_group(name=feature_group_name, version=version)
fg.insert(df_forecast_weather)

# Define and insert the Air Quality Feature Group
feature_group_name = "stockholm_air_quality"
version = 1
fg = fs.get_feature_group(name=feature_group_name, version=version)
fg.insert(df_forecast_aq)